In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from tqdm import tqdm

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class NN(nn.Module):
    def __init__(self, input_size, num_classes):
        super(NN, self).__init__()
        self.layer1 = nn.Linear(input_size, 50)
        self.layer2 = nn.Linear(50, num_classes)
        
    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = self.layer2(x)
        return x

In [13]:
# Hyperparams
input_size = 784 #28*28
num_classes =10
learning_rate = 0.001
batch_size = 100
num_epochs = 1

In [4]:
# MNIST Dataset
# Data Loading
train_dataset = datasets.MNIST(root='Data/MNIST-Dataset', train=True, transform=transforms.ToTensor(), download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_dataset = datasets.MNIST(root='Data/MNIST-Dataset', train=False, transform=transforms.ToTensor(), download=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

In [5]:
model = NN(input_size=input_size, num_classes=num_classes).to(device)
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [6]:
# for batch_idx, (data, targets) in enumerate(train_loader):
print(train_dataset[0])
print(len(train_dataset))

(tensor([[[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000

In [7]:
for batch_idx, (data, targets) in enumerate(train_loader):
    print(batch_idx)
    print(data.shape)
    print(targets.shape)
    # print(data[0])
    print()
    break

0
torch.Size([100, 1, 28, 28])
torch.Size([100])



In [14]:
model.train()  # Setting model into training mode

for epoch in range(num_epochs):
    num_samples = 0
    num_correct = 0
    loop_info = tqdm(enumerate(train_loader), total=len(train_loader), leave=False)
    for batch_idx, (data, targets) in loop_info:
        data = data.to(device=device)
        targets = targets.to(device=device)
        # Re-sizing data to feed NN
        data = data.reshape(data.shape[0], -1)
        
        # Output prediction
        predicted_output = model(data)
        
        # Calculate Accuracy
        _, prediction = predicted_output.max(1)
        num_samples += predicted_output.size(0)
        num_correct += (prediction==targets).sum()
        
        # calculate predicted output and loss
        loss = loss_function(predicted_output, targets)
        # Resetting gradients to zero and update weights based on gradients computed in loss.backward()
        optimizer.zero_grad()
        loss.backward()
        # Performing gradient descent
        optimizer.step()
        
        # Update progress bar
        acc = (num_correct / num_samples * 100.00)
        loop_info.set_description(f'Epoch [{epoch}/{num_epochs}]')
        loop_info.set_postfix(acc=acc)
        
#         if ((batch_idx+1) % 200 == 0):
#             print(f'Batch {batch_idx+1}: Accuracy: {num_correct / num_samples * 100.00}')
#     print(f'Epoch {epoch+1} completed with Accuracy {num_correct / num_samples * 100.00}')
#     print()
print()
print("Training Completed!")

        
   


Training Completed!


In [15]:
# Model Testing
def check_accuracy(loader, model):
    model.eval() # Set model into evaluation mode!
    num_samples = 0
    num_correct = 0
    for data, targets in loader:
            data = data.to(device=device)
            targets = targets.to(device=device)
            data = data.reshape(data.shape[0], -1)
            predicted_output = model(data)

            # Calculate Accuracy
            _, prediction = predicted_output.max(1)
            num_samples += predicted_output.size(0)
            num_correct += (prediction==targets).sum()
    
    print(f'Correctly identified samples: {num_correct}')
    print(f'Total samples: {num_samples}')
    print(f'The Validation Accuracy is {num_correct / num_samples * 100.00:.2f}')
    model.train() # Setting model back to training data

In [16]:
print("Checking Accuracy on Training Data...")
check_accuracy(train_loader, model)
print()
print("Checking Accuracy on Testing Data...")
check_accuracy(test_loader, model)

Checking Accuracy on Training Data...
Correctly identified samples: 57980
Total samples: 60000
The Validation Accuracy is 96.63

Checking Accuracy on Testing Data...
Correctly identified samples: 9615
Total samples: 10000
The Validation Accuracy is 96.15
